# setup

In [1]:
HOME = "/data/git/product-category"

In [2]:
# http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210304A1.ipynb
prfx_prp = 'prep_20210304A1'
p_out = f'{HOME}/data/transformer_20210305A1'
!mkdir -p {p_out}

In [3]:
import pandas as pd
import numpy as np
from collections import Counter

# eda 

In [4]:
%%time
# df = pd.read_csv(f'../data/data__{prfx_prp}.csv')
df = pd.read_csv(f'{HOME}/data/data_sample__{prfx_prp}.csv')

print(df.shape)
df.sample(3)

(10000, 9)
CPU times: user 134 ms, sys: 20.2 ms, total: 154 ms
Wall time: 152 ms


,category,description,title,brand,feature,asin,domain,txt,is_validation
2299,Electronics|Computers &amp; Accessories|Monitors,Acer's AL1714 17-inch LCD monitor features hig...,"Acer AL1714CB 17"" Flat Panel Display",Acer,"1,280 x 1,024 native resolution, 0.264mm dot p...",B0002JC2RI,Electronic,"Acer AL1714CB 17"" Flat Panel Display Acer Acer...",0
6807,Books|Engineering & Transportation|Transportation,Our Misc. Tractors Owatonna 2040 Mustang Skid ...,Misc. Tractors Owatonna 2040 Mustang Skid Stee...,Misc. Tractors Manuals,NaN,B007HMSR5E,Book,Misc. Tractors Owatonna 2040 Mustang Skid Stee...,1
6987,Home & Kitchen|Kitchen & Dining|Kitchen & Tabl...,"NEW IN BAG, ORIGINAL CHEFSKIN BRAND, REAL FABR...",Chefskin Chef Set (Blue Apron + 1 Adjustable W...,CHEFSKIN,"THIS IS THE BEST QUALITY KIDS APRONS AND HATS,...",B005MVRBN6,Home_and_Kitche,Chefskin Chef Set (Blue Apron + 1 Adjustable W...,0


In [5]:
MIN_CNT = 50
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter((j for i in df.category.apply(lambda x: x.split('|')) for j in i))
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

print("len(i2dmn), len(i2cat)", len(i2dmn), len(i2cat))
print("|".join(i2cat))

len(i2dmn), len(i2cat) 23 111
100% Cotton|100% Leather|100% Polyester|Accessories|Accessories & Supplies|Action Figures & Statues|Americas|Arts &amp; Photography|Arts, Crafts & Sewing|Athletic|Audio & Video Accessories|Automotive|Baby|Baby Girls|Bags, Cases & Sleeves|Basic Cases|Bedding|Biographies &amp; Memoirs|Body & Trim|Books|Boots|Boys|Bracelets|Bumper Stickers, Decals & Magnets|CDs & Vinyl|Camera & Photo|Cases, Holsters & Sleeves|Casual|Cell Phones & Accessories|Children's Books|Christian Books &amp; Bibles|Clothing|Clothing, Shoes & Jewelry|Computer Accessories & Peripherals|Computers & Accessories|Dining & Entertaining|Dogs|Dresses|Earrings|Electronics|Exercise & Fitness|Exterior Accessories|Fan Shop|Furniture|Games|Gardening & Lawn Care|Genre Fiction|Genre for Featured Categories|Girls|Grocery & Gourmet Food|Hand Tools|Handbags & Wallets|History|Home & Kitchen|Home Dcor|Home Dcor Accents|Humanities|Hunting & Fishing|Imported|Industrial & Scientific|Interior Accessories|Jewelry

In [6]:
## make ys

ys = np.zeros((len(df), len(i2cat)))

for i,cats in enumerate(df.category):
    idx_pos = [cat2i[cat] for cat in cats if cat in cat2i]
    ys[i,idx_pos] = 1

print("ys.shape", ys.shape)

ys.shape (10000, 111)


# modeling setup

- https://colab.research.google.com/drive/1F_RNcHzTfFuQf-LeKvSlud6x7jXYkG31#scrollTo=goRmGIRI5cfC

In [7]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from transformers.optimization import AdamW

# dataset

In [8]:
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer
import torch

In [9]:
def mk_tensors(txt, tokenizer, max_seq_length):
    tok_res = tokenizer(
        txt, truncation=True, padding='max_length', max_length=max_seq_length
    )
    input_ids = tok_res["input_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask

def mk_ds(txt, tokenizer, max_seq_length, ys):
    input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
    return TensorDataset(input_ids, 
                         attention_mask, 
                         torch.tensor(ys)) 

class PCDataModule(pl.LightningDataModule):
    def __init__(self, 
                 model_name_or_path, 
                 max_seq_length, 
                 min_products_for_category,
                 train_batch_size,
                 val_batch_size,
                 data_file_path=None,
                 dataframe=None):
        super().__init__()
        self.data_file_path = data_file_path
        self.dataframe = dataframe
        self.min_products_for_category = min_products_for_category
        self.model_name_or_path = model_name_or_path
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.num_classes = None
      
    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoTokenizer.from_pretrained(self.model_name_or_path)

    def setup(self, stage=None):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path)
        if self.dataframe is None:
            self.dataframe = pd.read_csv(self.data_file_path)
        cats = self.dataframe.category.apply(lambda x: x.split('|'))
        cat2cnt = Counter((j for i in cats for j in i))
        i2cat = sorted(k for k,v in cat2cnt.items() if v>self.min_products_for_category)
        cat2i = {v:k for k,v in enumerate(i2cat)}
        self.num_classes = len(i2cat)
        self.i2cat, self.cat2i = i2cat, cat2i
        
        ys = np.zeros((len(self.dataframe), len(i2cat)))
        for i,cats in enumerate(self.dataframe.category):
            idx_pos = [cat2i[cat] for cat in cats if cat in cat2i]
            ys[i,idx_pos] = 1
        
        msk_val = self.dataframe.is_validation==1
        self.df_trn = self.dataframe[~msk_val]
        self.df_val = self.dataframe[msk_val]
        idx_trn = np.where(~msk_val)[0]
        idx_val = np.where(msk_val)[0]
        self.ys_trn, self.ys_val = ys[idx_trn], ys[idx_val]
        
        txt = self.dataframe.txt.values
        self.train_dataset = mk_ds(list(self.df_trn.txt), self.tokenizer, self.max_seq_length, self.ys_trn)
        self.eval_dataset  = mk_ds(list(self.df_val.txt), self.tokenizer, self.max_seq_length, self.ys_val)
        
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
        )

    def val_dataloader(self):
        return DataLoader(
            self.eval_dataset,
            batch_size=self.val_batch_size,
        )

# model

In [19]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


class PCModel(pl.LightningModule):
    def __init__(self, model_name_or_path, num_classes, learning_rate, adam_beta1, adam_beta2, adam_epsilon):
        super().__init__()
        self.save_hyperparameters()
        self.model_name_or_path = model_name_or_path
        self.bert = AutoModel.from_pretrained(self.model_name_or_path)
        self.num_classes = num_classes
        self.W = nn.Linear(self.bert.config.hidden_size, self.num_classes)

    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoModel.from_pretrained(self.model_name_or_path)

    def forward(self, input_ids, attention_mask):
        h = self.bert(input_ids, attention_mask)['last_hidden_state']
        h_cls = h[:, 0]
        return self.W(h_cls)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        self.log('valid_loss', loss, on_step=True, sync_dist=True)
        return {'val_loss': loss}
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(),
                          self.hparams.learning_rate,
                          betas=(self.hparams.adam_beta1,
                                 self.hparams.adam_beta2),
                          eps=self.hparams.adam_epsilon,)
        return optimizer    
    
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=5e-5)
        parser.add_argument('--adam_beta1', type=float, default=0.9)
        parser.add_argument('--adam_beta2', type=float, default=0.999)
        parser.add_argument('--adam_epsilon', type=float, default=1e-8)
        return parser

In [20]:
parser = ArgumentParser()

parser.add_argument('--model_name_or_path', type=str,
                    default="distilbert-base-cased")
parser.add_argument('--max_seq_length', type=int, default=32)
parser.add_argument('--min_products_for_category', type=int, default=100)
parser.add_argument('--train_batch_size', type=int, default=16)
parser.add_argument('--val_batch_size', type=int, default=8)

parser = pl.Trainer.add_argparse_args(parser)
parser = PCModel.add_model_specific_args(parser)

args = parser.parse_args([
    '--default_root_dir', p_out,
])


data_module = PCDataModule(
    model_name_or_path=args.model_name_or_path,
    data_file_path=f'{HOME}/data/data_sample__{prfx_prp}.csv',
    min_products_for_category=args.min_products_for_category,
    max_seq_length=args.max_seq_length,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
)

data_module.prepare_data()

In [21]:
%%time
data_module.setup()

CPU times: user 9.54 s, sys: 74.3 ms, total: 9.62 s
Wall time: 1.82 s


In [22]:
pcmodel = PCModel(
    model_name_or_path=args.model_name_or_path,
    num_classes= data_module.num_classes,
    learning_rate=args.learning_rate,
    adam_beta1=args.adam_beta1,
    adam_beta2=args.adam_beta2,
    adam_epsilon=args.adam_epsilon,
)
pcmodel.prepare_data()

In [23]:
pl.seed_everything(1234)


# trainer = pl.Trainer.from_argparse_args(args)
# trainer = pl.Trainer.from_argparse_args(args, fast_dev_run=True)
trainer = pl.Trainer.from_argparse_args(args, fast_dev_run=True, gpus=1)
# trainer = pl.Trainer.from_argparse_args(args, limit_train_batches=10, limit_val_batches=5, gpus=1)
# trainer = pl.Trainer.from_argparse_args(args, gpus=1)
trainer.fit(pcmodel, data_module)


Global seed set to 1234
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).

  | Name | Type            | Params
-----------------------------------------
0 | bert | DistilBertModel | 65.2 M
1 | W    | Linear          | 40.8 K
-----------------------------------------
65.2 M    Trainable params
0         Non-trainable params
65.2 M    Total params
260.927   Total estimated model params size (MB)
/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightnin

/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


1

In [25]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 16306), started 0:10:21 ago. (Use '!kill 16306' to kill it.)

In [34]:
!nvidia-smi

Fri Mar  5 13:55:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    38W / 300W |   3675MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  